# 1. Import pacakage

In [1]:
# conda install -n sim_search ipykernel --update-deps --force-reinstall
import pandas as pd
import numpy as np

from utils.search import return_res
from utils.data_process import input_process, res_process
from utils.search import do_search
from utils.plot import plotly_plot
# import matplotlib.pyplot as plt
import talib
from plotly.subplots import make_subplots
import plotly.graph_objects as go


# 2. read data

In [2]:
df = pd.read_csv('./data/2330TW.csv')
df.head()
# df.rename({'Unnamed: 0': 'Time'}, axis=1, inplace=True)

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits
0,2008-01-02,36.155183,36.155183,35.105517,35.280460,39300705,0.0,0.0
1,2008-01-03,34.930569,34.988883,34.638996,34.638996,35123067,0.0,0.0
2,2008-01-04,34.580682,35.222144,34.114164,34.988884,48851900,0.0,0.0
3,2008-01-07,33.239444,33.822592,32.539665,32.539665,104088436,0.0,0.0
4,2008-01-08,32.539672,33.006191,32.364728,32.656303,47704464,0.0,0.0


# 3. Input data proccess

In [3]:
s = 1604# import your time here
e = 1628

In [4]:
input_proccess = input_process.full_wav_scalar(full_wav=np.array(df['Close']), start_p=s, end_p=e, scalar_mehod="min_max")
full_wav_std = input_proccess.apply_wav()
baseline_wav_std = full_wav_std[s:e]

# 4. Do algo search

In [5]:
# algo_li = ["shape_d", "mink_d", "corr_d", "dtw_d", "poly_d"] # 放定義好得相似度
algo_li = ["shape_d", "mink_d", "corr_d", "dtw_d", "poly_d", "polycos_d"] 
res_d = {}

for alg in algo_li:
    search = do_search.search(baseline_wav=baseline_wav_std, full_wav=full_wav_std)
    res_d['{}'.format(alg)] = search.run_search_test(algo_option=alg)

100%|██████████| 3552/3552 [00:55<00:00, 63.70it/s]


In [40]:
polycos_d_sorted 

{'1604:1628': 1.0,
 '1554:1578': 0.9999514355692578,
 '2901:2925': 0.9999419186284277,
 '2980:3004': 0.9999111010717583,
 '1910:1934': 0.9998682401751564,
 '1659:1683': 0.9998602845656792,
 '3397:3421': 0.9998426170330413,
 '1547:1571': 0.9997344572287912,
 '87:111': 0.9996771385414811,
 '1051:1075': 0.9996237140570162,
 '2264:2288': 0.9996143292483725,
 '1180:1204': 0.9996045770644635,
 '2472:2496': 0.9995987060525717,
 '3488:3512': 0.9995879683001224,
 '986:1010': 0.9995777914271142,
 '3476:3500': 0.9995745650252377,
 '686:710': 0.9995725147266898,
 '3283:3307': 0.999568065288492,
 '1243:1267': 0.9995526121589153,
 '33:57': 0.9995509308128698,
 '1442:1466': 0.9995380541026487,
 '1777:1801': 0.9994702127955133,
 '3268:3292': 0.9994640548572478,
 '1747:1771': 0.9994545384501208,
 '16:40': 0.9994349883915805,
 '1501:1525': 0.9994216031263283,
 '3154:3178': 0.9993269605428906,
 '2587:2611': 0.9993198530856469,
 '2625:2649': 0.9993098006462331,
 '2974:2998': 0.9992982195019111,
 '2338:236

# 5. Dictionay result intersection 

In [37]:
# sort dict 
mink_d_sorted = dict(sorted(res_d['mink_d'].items(), key=lambda item: item[1])) # 小到大
corr_d_sorted = dict(sorted(res_d['corr_d'].items(), key=lambda item: item[1], reverse=True)) # 大到小
dtw_d_sorted = dict(sorted(res_d['dtw_d'].items(), key=lambda item: item[1]))
shape_d_sorted = dict(sorted(res_d['shape_d'].items(), key=lambda item: item[1]))
poly_d_sorted = dict(sorted(res_d['poly_d'].items(), key=lambda item: item[1]))
polycos_d_sorted = dict(sorted(res_d['polycos_d'].items(), key=lambda item: item[1]))

### 5.1 單一結果

In [13]:
# x = return_res.get_sim_res(df, s, e, "corr_d")
# print(x)

In [38]:
clear_res_engine = res_process.clean_duplicate_li(list(polycos_d_sorted.keys())[0:20])
print(clear_res_engine.ret_uq_range())

['1554:1578', '2901:2925', '2980:3004', '1910:1934', '1659:1683', '3397:3421', '87:111', '1051:1075', '2264:2288', '1180:1204', '2472:2496', '3488:3512', '986:1010', '686:710', '3283:3307', '1243:1267', '33:57']


In [32]:
clear_res_engine = res_process.clean_duplicate_li(list(poly_d_sorted.keys())[0:20])
print(clear_res_engine.ret_uq_range())

['2475:2499', '3142:3166', '2980:3004', '1706:1730', '2940:2964', '3116:3140', '2605:2629', '1990:2014', '2428:2452', '1180:1204', '165:189', '894:918', '2670:2694', '2830:2854', '3241:3265', '360:384', '237:261']


In [16]:
clear_res_engine = res_process.clean_duplicate_li(list(corr_d_sorted.keys())[0:20])
print(clear_res_engine.ret_uq_range())


['730:754', '3178:3202', '2602:2626', '341:365', '1546:1570', '298:322', '1320:1344', '1138:1162', '2840:2864', '1968:1992', '2074:2098', '1433:1457', '2002:2026', '1402:1426', '879:903']


In [17]:
clear_res_engine = res_process.clean_duplicate_li(list(mink_d_sorted.keys())[0:20])
print(clear_res_engine.ret_uq_range())

['1138:1162', '1546:1570', '1320:1344', '2074:2098', '1968:1992', '1360:1384', '460:484', '2839:2863', '1042:1066', '1433:1457', '1402:1426', '730:754']


In [18]:
clear_res_engine = res_process.clean_duplicate_li(list(dtw_d_sorted.keys())[0:50])
print(clear_res_engine.ret_uq_range())

['3138:3162', '1135:1159', '2073:2097', '763:787', '494:518', '1547:1571', '2841:2865', '3005:3029', '1402:1426', '1968:1992', '881:905', '1433:1457', '2271:2295', '2776:2800']


In [19]:
clear_res_engine = res_process.clean_duplicate_li(list(shape_d_sorted.keys())[0:10])
print(clear_res_engine.ret_uq_range())

['341:365', '2933:2957', '1869:1893', '1373:1397', '820:844', '3259:3283', '421:445', '2980:3004', '631:655']


### 5.2 多結果合併

In [20]:
# get intersection
# priority dict放前面
res_process.res_d_set(d1 = corr_d_sorted, d2 = mink_d_sorted, rank_d1=5, rank_d2=5)

set()

In [21]:
print(s,e)

1604 1628


# Result Visualization

In [24]:
# s,e = 2330, 2380
f1 = plotly_plot.plot(df, s = s, e= e, step=5)
# print(s,e)

In [25]:
f1

In [4]:
from dash import dcc

In [26]:
# 2138:2193 # 103:158 # 320:375 # 4422:4472 # 1212:1262 # 1150:1200 # 4940:4990 # 1100:1123 # 4598:4621 #2604:2627 #460:483 #4729:4752 # 3931:3954
f2 = plotly_plot.plot(df, s = 2475, e= 2499, step=3) #3547:3570 #19 42 # 38 61 #1551:1574 # 1942:1992 #3324:3374 # 1859:1909 # 1057:1107 # 1199:1269 # 1131:1152' # 2730:2751 # 1130:1151
f2 # 1155:1176 # 1130:1151 #1895:1916 # 1857:1878 # 2239:2260 #2417:2471 #2239:2260 @1131:1152 #2475:2499 # 1554:1578

In [39]:
# 978:1002
f2 = plotly_plot.plot(df, s =1554, e= 1578, step=3)   
f2 # 1554:1578
# 730:754
#1554:1578'

In [34]:
def figures_to_html(figs, filename="dashboard.html"):   
    with open(filename, 'w') as dashboard:
        dashboard.write("<html><head></head><body>" + "\n")
        for fig in figs:
            inner_html = fig.to_html().split('<body>')[1].split('</body>')[0]
            dashboard.write(inner_html)
        dashboard.write("</body></html>" + "\n")

In [35]:
figures_to_html([f1,f2])